In [58]:
import pandas as pd
from datetime import datetime, timedelta

file_path = '/content/drive/MyDrive/RozsAgloSP4-2018-vystup.xlsx'
sheet_name = 'Sheet1'
dataset = pd.read_excel(file_path, sheet_name=sheet_name)


In [59]:
dataset

,Zapis merania,Zmena,8hod,12hod,SP,Vzorka,Zhoda,0-5,10-25,nad 25,nad 10,Upravené časy intervalov,Vykonanie merania,Adjusted Time Intervals
0,2018-12-31 12:08:18,1,1D,R7,4,41679,A,10.71,42.50,25.36,67.86,2018-12-31 11:00:00,2018-12-31 11:00:18,2018-12-31 11:00:00
1,2018-12-30 23:56:11,3,3E,N9,4,41676,A,6.90,37.93,34.48,72.41,2018-12-30 23:00:00,2018-12-30 23:00:11,2018-12-30 23:00:00
2,2018-12-30 12:43:11,1,1D,R8,4,41673,A,8.05,38.98,30.08,69.06,2018-12-30 11:00:00,2018-12-30 11:00:11,2018-12-30 11:00:00
3,2018-12-30 00:15:03,3,3B,N5,4,41670,A,6.21,46.58,28.57,75.15,2018-12-29 23:00:00,2018-12-29 23:00:03,2018-12-29 23:00:00
4,2018-12-29 12:23:16,1,1A,R9,4,41667,A,4.21,42.11,34.74,76.85,2018-12-29 11:00:00,2018-12-29 11:00:16,2018-12-29 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,2018-01-03 01:58:37,3,3D,N6,4,40006,A,7.04,40.61,33.57,74.18,2018-01-02 23:00:00,2018-01-02 23:00:37,2018-01-02 23:00:00
498,2018-01-02 11:30:07,1,1B,R5,4,40004,N,36.17,26.60,15.96,42.56,2018-01-02 11:00:00,2018-01-02 11:00:07,2018-01-02 11:00:00
499,2018-01-02 02:02:57,3,3D,N7,4,40003,A,11.62,29.05,28.63,57.68,2018-01-01 23:00:00,2018-01-01 23:00:57,2018-01-01 23:00:00
500,2018-01-01 16:49:22,2,2C,R6,4,40002,A,9.46,40.19,33.80,73.99,2018-01-01 11:00:00,2018-01-01 11:00:22,2018-01-01 11:00:00


In [60]:
adjusted_times = []
# Iterating through all original dates and measurement times
for time in dataset['Zapis merania']:

    time_str = time.strftime("%Y-%m-%d %H:%M:%S")
    date_time = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")

    # Adjusting the measurement time
    if date_time.hour >= 23:
        date_time = date_time.replace(hour=23, minute=0)
    elif date_time.hour >= 11:
        date_time = date_time.replace(hour=11, minute=0)
    else:
        date_time -= timedelta(days=1)
        date_time = date_time.replace(hour=23, minute=0)
    # Adding the adjusted date and time to a new list
    adjusted_times.append(date_time.strftime("%Y-%m-%d %H:%M"))

dataset['Adjusted Time Intervals'] = adjusted_times

# Converting the 'Adjusted Time Intervals' column to datetime type
dataset['Adjusted Time Intervals'] = pd.to_datetime(dataset['Adjusted Time Intervals'])
separate_list = []
groups = dataset.groupby('Adjusted Time Intervals')

for _, group in groups:
    # Getting the date and time for the first row in the group
    first_time = group.iloc[0]['Zapis merania']
    nearest_time = first_time

    for _, row in group.iterrows():
        # Getting the date and time for the current row
        current_time = row['Zapis merania']
        # Comparing the current time with the nearest found time
        if abs((current_time - row['Adjusted Time Intervals']).total_seconds()) < abs((nearest_time - row['Adjusted Time Intervals']).total_seconds()):
            nearest_time = current_time
    # Adding the nearest time to a separate list
    separate_list.append(nearest_time)

In [61]:
# Filtering rows whose time matches the times in the separate_list
adjusted_data = dataset[dataset['Zapis merania'].isin(separate_list)]
adjusted_data = adjusted_data.reset_index(drop=True)

adjusted_data.to_excel('/content/drive/MyDrive/RozsAgloSP4-2018-vystup.xlsx', index=False)


In [62]:
file_path = '/content/drive/MyDrive/RozsAgloSP4-2018-vystup.xlsx'
data = pd.read_excel(file_path, 'Sheet1')
data
file_path1 = '/content/drive/MyDrive/PI_DataLink_Aglo_digital twins-5min-SP4-2018-15minute.xlsm'
sheet_name1 = 'Sheet1'
data1 = pd.read_excel(file_path1, sheet_name=sheet_name1)

In [63]:
data1 = data1.iloc[3:, 2:]
data1 = data1.reset_index(drop=True)

In [65]:
# Convert columns to date type
data['Dátum'] = pd.to_datetime(data['Upravené časy intervalov']).dt.date
data1['Dátum'] = pd.to_datetime(data1['Unnamed: 2']).dt.date

# Finding matching dates
result = data1[data1['Dátum'].isin(data['Dátum'])]
# Renaming columns
result = result.rename(columns={'Unnamed: 2': 'Interval od', 'Unnamed: 3': 'Interval do'})
result = result.drop_duplicates(subset=['Interval od'])

result['Zapis merania'] = None
result['Vykonanie merania'] = None
result['Vzorka'] = None
result['Zhoda'] = None
result['Rozsev 0-5'] = None

for index, row in result.iterrows():
    interval_od = row['Interval od']
    interval_do = row['Interval do']
    mask = (data['Upravené časy intervalov'] >= interval_od) & (data['Upravené časy intervalov'] < interval_do)
    filtered_data = data[mask]
    if not filtered_data.empty:
        result.loc[index, 'Zapis merania'] = filtered_data['Vykonanie merania'].iloc[0]
        result.loc[index, 'Vykonanie merania'] = filtered_data['Upravené časy intervalov'].iloc[0]
        result.loc[index, 'Vzorka'] = filtered_data['Vzorka'].iloc[0]
        result.loc[index, 'Zhoda'] = filtered_data['Zhoda'].iloc[0]
        result.loc[index, 'Rozsev 0-5'] = filtered_data['0-5'].iloc[0]

result = result.drop('Dátum', axis=1)
result = result.drop('výstupné dáta, látkové filtre - pred komínom', axis=1)
result = result.drop('vstupné dáta, 3 stupeň', axis=1)
result = result.drop('Unnamed: 12', axis=1)
result = result.drop('Unnamed: 13', axis=1)
result = result.drop('Unnamed: 14', axis=1)
result = result.drop('Unnamed: 16', axis=1)
result = result.drop('výstupné dáta.3', axis=1)
result = result.drop('výstupné dáta.4', axis=1)
result = result.drop('Unnamed: 27', axis=1)
result = result.drop('výstupné dáta.5', axis=1)
result = result.drop('Unnamed: 30', axis=1)
result = result.drop('vstupné dáta.2', axis=1)
result = result.drop('vstupné dáta.3', axis=1)
result = result.drop('vstupné dáta, 3 stupen miesania, reguluje palič', axis=1)
result = result.drop('vstupné dáta, reguluje palič.2', axis=1)
result = result.drop('vstupné dáta', axis=1)
result = result.drop('vstupné dáta, reguluje palič.10', axis=1)


In [66]:
result = result.reset_index(drop=True)
output = '/content/output.xlsx'
result.to_excel(output, index=False)



In [67]:
new_file = pd.read_excel(output)
new_file = new_file[['Interval od', 'Interval do','Zapis merania', 'Vykonanie merania', 'Vzorka', 'Zhoda', 'Rozsev 0-5']]
meranie_interval = pd.concat([new_file, data1['výstupné dáta.5']], axis=1)
meranie_interval.to_excel('rozsev_v_intervaloch.xlsx', index=False)


In [68]:
# Removing specified columns from result and disregarding indexing
dataset_final = result.drop(columns=['Interval od', 'Interval do', 'Zapis merania', 'Vykonanie merania', 'Vzorka', 'Zhoda'])
dataset_final.dropna(inplace=True)
dataset_final.drop_duplicates(subset=['Rozsev 0-5'])
dataset_final = dataset_final.reset_index(drop=True)


In [69]:
# Saving to file
dataset_final.to_excel("dataset.xlsx", index=False)
dataset_final

,výstupné dáta,výstupné dáta.1,výstupné dáta.2,"vstupné dáta, reguluje palič","vstupné dáta, reguluje palič.1","vstupné dáta, reguluje palič.3","vstupné dáta, reguluje palič.4","vstupné dáta, reguluje palič.5","vstupné dáta, reguluje palič.6","vstupné dáta, reguluje palič.7",...,výstupné dáta.9,výstupné dáta.10,výstupné dáta.11,výstupné dáta.12,výstupné dáta.13,výstupné dáta.14,výstupné dáta.15,vstupné dáta.1,"vstupné dáta, reguluje palič.12",Rozsev 0-5
0,18.245233,708.250231,26.974017,227.713533,3.7711,3.22,-4.185,-4.163883,-4.975017,-4.275133,...,121.342817,165.478184,166.134733,196.306917,220.87265,192.920016,129.854734,0.740317,1.979583,9.46
1,17.924417,718.155392,31.431067,227.747202,3.567233,3.13,-4.577833,-4.622817,-5.482067,-4.6863,...,116.6768,161.837916,161.375434,186.621433,214.378767,186.251648,134.829633,0.761467,2.018433,11.62
2,19.169966,709.452829,161.9624,226.832703,3.276133,3,-4.9038,-4.996,-5.932367,-5.116333,...,71.735,101.088067,94.693117,112.700217,125.127517,105.031466,70.711683,0.765667,2.047617,36.17
3,18.356667,709.410084,61.53855,227.996302,3.473383,3.09,-4.315217,-4.344033,-5.166083,-4.456683,...,139.248766,177.915884,176.855198,204.440951,234.291532,208.389883,140.35035,0.748317,2.00315,7.04
4,18.993284,715.636641,78.977666,190.246086,2.574417,2.86,-4.38765,-4.3904,-5.196567,-4.473067,...,123.216516,156.741516,138.683849,168.911166,180.099066,156.663517,105.52515,0.702717,1.9422,7.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,18.816999,710.190049,81.033282,227.547802,4.6434,3.9,-3.463567,-4.221117,-5.001833,-4.300567,...,171.417384,156.030383,151.898268,128.774884,170.230866,177.4666,147.93215,0.66065,1.824967,4.21
497,18.816999,706.556821,72.524867,228.063869,4.7744,3.81,-3.551733,-4.48795,-5.244917,-4.626683,...,141.585617,147.590018,152.835733,134.094401,177.335335,183.941018,147.457733,0.7289,2.113767,6.21
498,18.816999,695.341931,64.947766,227.116967,4.442967,3.67,-3.978633,-4.990883,-5.74105,-5.123233,...,141.632734,132.0885,132.520417,117.497401,166.51765,184.331782,161.278251,0.712333,2.08055,8.05
499,18.816999,707.283079,77.876267,226.86465,4.131533,3.53,-3.697167,-4.638483,-5.393767,-4.82605,...,137.372949,132.912383,142.0003,139.437549,204.874482,220.287483,200.544516,0.749917,2.152633,6.9
